In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [7]:
df=pd.read_csv('CKD.csv')
df=pd.get_dummies(df,dtype=int,drop_first=True)
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [9]:
df.isnull().values.any()

np.False_

In [10]:
indep=df[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=df['classification_yes']

Naive_bayes


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5] 
   }
grid=GridSearchCV(GaussianNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]},
             scoring='f1_weighted', verbose=3)

In [12]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)

[[149   1]
 [  4 245]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       150
           1       1.00      0.98      0.99       249

    accuracy                           0.99       399
   macro avg       0.98      0.99      0.99       399
weighted avg       0.99      0.99      0.99       399

The f1_macro value for the best parameters {'var_smoothing': 1e-09}: 0.9874927342358296
The roc_score: 0.996532797858099


In [13]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012959,0.009179,0.008232,0.002096,1.000000e-09,{'var_smoothing': 1e-09},0.962813,0.987539,0.987539,1.000000,0.974684,0.982515,0.012694,1
1,0.009155,0.001438,0.011446,0.008320,1.000000e-08,{'var_smoothing': 1e-08},0.852273,0.950521,0.962813,0.975148,0.974828,0.943117,0.046319,2
2,0.005848,0.001018,0.006352,0.001540,1.000000e-07,{'var_smoothing': 1e-07},0.637626,0.876984,0.926015,0.962813,0.962330,0.873154,0.121880,3
3,0.006128,0.000940,0.009146,0.007525,1.000000e-06,{'var_smoothing': 1e-06},0.607143,0.876984,0.889274,0.938259,0.962330,0.854798,0.127705,4
4,0.006990,0.002690,0.006118,0.002181,1.000000e-05,{'var_smoothing': 1e-05},0.622527,0.852273,0.876984,0.889143,0.887646,0.825715,0.102448,5


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import BernoulliNB
param_grid = { 'alpha': [0.5, 1.0, 1.5, 2.0], 
    'binarize': [0.0, 0.5, 1.0]  
   }
grid=GridSearchCV(BernoulliNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.5, 1.0, 1.5, 2.0],
                         'binarize': [0.0, 0.5, 1.0]},
             scoring='f1_weighted', verbose=3)

In [15]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)

[[149   1]
 [  7 242]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       150
           1       1.00      0.97      0.98       249

    accuracy                           0.98       399
   macro avg       0.98      0.98      0.98       399
weighted avg       0.98      0.98      0.98       399

The f1_macro value for the best parameters {'alpha': 0.5, 'binarize': 0.5}: 0.9800241876810629
The roc_score: 0.9948862115127175


In [16]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_binarize,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005930,0.001077,0.005185,0.000399,0.5,0.0,"{'alpha': 0.5, 'binarize': 0.0}",0.913778,0.950521,0.987539,1.000000,0.974684,0.965304,0.030528,2
1,0.005656,0.000558,0.006346,0.001421,0.5,0.5,"{'alpha': 0.5, 'binarize': 0.5}",0.913778,0.975148,0.987539,1.000000,0.974684,0.970230,0.029721,1
2,0.004954,0.000391,0.005030,0.000534,0.5,1.0,"{'alpha': 0.5, 'binarize': 1.0}",0.775000,0.820231,0.870014,0.687046,0.922827,0.815024,0.080812,9
3,0.004829,0.000661,0.004640,0.001507,1.0,0.0,"{'alpha': 1.0, 'binarize': 0.0}",0.913778,0.938259,0.975148,0.950521,0.974684,0.950478,0.023198,3
4,0.004673,0.000160,0.005288,0.001395,1.0,0.5,"{'alpha': 1.0, 'binarize': 0.5}",0.913778,0.938259,0.975148,0.950521,0.974684,0.950478,0.023198,3
5,0.004588,0.000488,0.004271,0.000761,1.0,1.0,"{'alpha': 1.0, 'binarize': 1.0}",0.775000,0.820231,0.870014,0.687046,0.922827,0.815024,0.080812,9
6,0.005156,0.001693,0.004699,0.000998,1.5,0.0,"{'alpha': 1.5, 'binarize': 0.0}",0.889274,0.938259,0.975148,0.950521,0.974684,0.945577,0.031527,5
7,0.004651,0.000630,0.004188,0.000399,1.5,0.5,"{'alpha': 1.5, 'binarize': 0.5}",0.889274,0.938259,0.975148,0.950521,0.974684,0.945577,0.031527,5
8,0.004300,0.000599,0.005161,0.000949,1.5,1.0,"{'alpha': 1.5, 'binarize': 1.0}",0.775000,0.820231,0.870014,0.687046,0.922827,0.815024,0.080812,9
9,0.006501,0.002645,0.007441,0.002835,2.0,0.0,"{'alpha': 2.0, 'binarize': 0.0}",0.889274,0.926015,0.950521,0.950521,0.962141,0.935694,0.026029,8


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
param_grid = { 'alpha': [0.5, 1.0, 1.5, 2.0] 
   }
grid=GridSearchCV(MultinomialNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.5, 1.0, 1.5, 2.0]}, scoring='f1_weighted',
             verbose=3)

In [18]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)

[[145   5]
 [ 49 200]]
              precision    recall  f1-score   support

           0       0.75      0.97      0.84       150
           1       0.98      0.80      0.88       249

    accuracy                           0.86       399
   macro avg       0.86      0.88      0.86       399
weighted avg       0.89      0.86      0.87       399

The f1_macro value for the best parameters {'alpha': 0.5}: 0.8667587676290408
The roc_score: 0.9545381526104418


In [19]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004787,0.000539,0.004890,0.000198,0.5,{'alpha': 0.5},0.672764,0.889274,0.913563,0.926015,0.924484,0.865220,0.097122,1
1,0.005083,0.000322,0.005686,0.000731,1.0,{'alpha': 1.0},0.658856,0.864654,0.913563,0.913778,0.924484,0.855067,0.100278,2
2,0.006205,0.000444,0.005214,0.000747,1.5,{'alpha': 1.5},0.644737,0.864654,0.913563,0.901535,0.924484,0.849795,0.104491,3
3,0.004460,0.000828,0.004390,0.000799,2.0,{'alpha': 2.0},0.644737,0.864654,0.889143,0.901535,0.924484,0.844911,0.101935,4


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import CategoricalNB
param_grid = { 'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]
   }
grid=GridSearchCV(CategoricalNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\Hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]},
             scoring='f1_weighted', verbose=3)

In [21]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)


[[150   0]
 [  1 248]]
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       150
           1       1.00      1.00      1.00       249

    accuracy                           1.00       399
   macro avg       1.00      1.00      1.00       399
weighted avg       1.00      1.00      1.00       399

The f1_macro value for the best parameters {'alpha': 0.1}: 0.9974953761738116
The roc_score: 1.0


In [22]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012021,0.000908,0.006493,0.001378,0.1,{'alpha': 0.1},0.962618,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.013694,0.004596,0.008317,0.004444,0.5,{'alpha': 0.5},0.962618,NaN,NaN,NaN,NaN,NaN,NaN,1
2,0.011122,0.000685,0.005490,0.002005,1.0,{'alpha': 1.0},0.950296,NaN,NaN,NaN,NaN,NaN,NaN,1
3,0.011410,0.001564,0.003299,0.000878,1.5,{'alpha': 1.5},0.950296,NaN,NaN,NaN,NaN,NaN,NaN,1
4,0.008896,0.001201,0.002702,0.001532,2.0,{'alpha': 2.0},0.950296,NaN,NaN,NaN,NaN,NaN,NaN,1


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import ComplementNB
param_grid = { 'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
              'norm': [True, False]
   }
grid=GridSearchCV(ComplementNB(),param_grid,refit = True, verbose = 3, n_jobs=-1, scoring='f1_weighted')
grid.fit(indep,dep)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(estimator=ComplementNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0],
                         'norm': [True, False]},
             scoring='f1_weighted', verbose=3)

In [24]:
re=grid.cv_results_
y_pred=grid.predict(indep)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(dep,y_pred)
print(cm)

from sklearn.metrics import classification_report
cr=classification_report(dep,y_pred)
print(cr)

from sklearn.metrics import f1_score
fs=f1_score(dep,y_pred,average='weighted')
print("The f1_macro value for the best parameters {}:".format(grid.best_params_),fs)

from sklearn.metrics import roc_auc_score
roc_score=roc_auc_score(dep,grid.predict_proba(indep)[:,1])
print("The roc_score:",roc_score)

[[145   5]
 [ 39 210]]
              precision    recall  f1-score   support

           0       0.79      0.97      0.87       150
           1       0.98      0.84      0.91       249

    accuracy                           0.89       399
   macro avg       0.88      0.91      0.89       399
weighted avg       0.91      0.89      0.89       399

The f1_macro value for the best parameters {'alpha': 0.1, 'norm': False}: 0.8912968721618211
The roc_score: 0.9623025435073628


In [25]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_norm,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005233,0.000685,0.004486,0.000686,0.1,True,"{'alpha': 0.1, 'norm': True}",0.204545,0.204545,0.204545,0.204545,0.209035,0.205443,0.001796,6
1,0.006451,0.001853,0.008524,0.005951,0.1,False,"{'alpha': 0.1, 'norm': False}",0.700000,0.901535,0.938022,0.926015,0.936901,0.880495,0.091197,1
2,0.005539,0.000394,0.004456,0.000665,0.5,True,"{'alpha': 0.5, 'norm': True}",0.204545,0.204545,0.204545,0.204545,0.209035,0.205443,0.001796,6
3,0.004337,0.000434,0.022237,0.014737,0.5,False,"{'alpha': 0.5, 'norm': False}",0.672764,0.889274,0.913563,0.926015,0.924484,0.865220,0.097122,2
4,0.005841,0.000651,0.023021,0.014970,1.0,True,"{'alpha': 1.0, 'norm': True}",0.204545,0.204545,0.204545,0.204545,0.209035,0.205443,0.001796,6
5,0.005137,0.000733,0.011358,0.008479,1.0,False,"{'alpha': 1.0, 'norm': False}",0.658856,0.864654,0.901354,0.901535,0.924484,0.850177,0.097562,3
6,0.004957,0.000538,0.004923,0.000342,1.5,True,"{'alpha': 1.5, 'norm': True}",0.204545,0.204545,0.204545,0.204545,0.209035,0.205443,0.001796,6
7,0.004395,0.000373,0.004799,0.001332,1.5,False,"{'alpha': 1.5, 'norm': False}",0.644737,0.864654,0.889143,0.901535,0.924484,0.844911,0.101935,4
8,0.007289,0.007330,0.003910,0.001061,2.0,True,"{'alpha': 2.0, 'norm': True}",0.204545,0.204545,0.204545,0.204545,0.209035,0.205443,0.001796,6
9,0.005347,0.001716,0.004125,0.001898,2.0,False,"{'alpha': 2.0, 'norm': False}",0.644737,0.864654,0.889143,0.901535,0.912103,0.842434,0.100106,5
